In [1]:
import pandas as pd

In [2]:
def format_res(df: pd.DataFrame, n_splits: int, cluster_outliers: str):

    precision_mean = df['precision'].mean()
    precision_se = df['precision'].sem()

    recall_mean = df['recall'].mean() 
    recall_se = df['recall'].sem()

    f1_mean = df['f1'].mean() 
    f1_se = df['f1'].sem()

    time_mean = df['time'].mean()
    time_se = df['time'].sem()

    output = (
        n_splits, 
        cluster_outliers,
        precision_mean,
        precision_se,
        recall_mean,
        recall_se,
        f1_mean,
        f1_se,
        time_mean,
        time_se
        )
    return output

In [3]:
def get_res(n_splits, cluster_outliers):
    df = pd.read_csv(f'res_dm_{n_splits}_{cluster_outliers}.csv')
    return format_res(df, n_splits, cluster_outliers)

In [4]:
df_res = pd.DataFrame(columns=['n_splits', 'cluster_outliers', 'precision_mean', 'precision_se', 'recall_mean', 'recall_se', 'f1_mean', 'f1_se', 'time_mean', 'time_se'])

# for n_splits in [2, 3, 4]:
for n_splits in range(5, 11):
    # for cluster_outliers in ['all', 'skip']:
    for cluster_outliers in ['all']:
        df_res.loc[len(df_res)] = get_res(n_splits, cluster_outliers)

In [5]:
df_res.sort_values(by=['precision_mean', 'f1_mean'], ascending=False)

,n_splits,cluster_outliers,precision_mean,precision_se,recall_mean,recall_se,f1_mean,f1_se,time_mean,time_se
1,6,all,0.972014,0.000783,0.904537,0.002992,0.937031,0.001426,17.348714,0.098614
0,5,all,0.971565,0.000741,0.890830,0.003101,0.929429,0.001938,17.431959,0.071766
2,7,all,0.969755,0.000798,0.912548,0.002510,0.940261,0.001222,17.007977,0.045317
3,8,all,0.969670,0.000917,0.916313,0.001709,0.942228,0.001025,16.805703,0.043628
4,9,all,0.969582,0.000643,0.919788,0.001022,0.944024,0.000563,16.895227,0.111103
5,10,all,0.969075,0.000401,0.919402,0.001089,0.943580,0.000498,17.115429,0.043221


For reference:

Basiline + 1by1 | t_s for 1by1 

* n_splits = 10 --> 30372 + 3375 | 61.696611
* n_splits = 50 --> 33072 + 675 | 14.362957
* n_splits = 100 --> 33409 + 338 | 7.342449

In [6]:
def get_splits(n_splits):
    if n_splits == 2:
        return '16874x2'
    elif n_splits == 3:
        return '11249x3'
    elif n_splits == 4:
        return '8437x4'

df_res['splits'] = df_res['n_splits'].apply(lambda x: get_splits(x))

In [7]:
df_res["precision"] = df_res["precision_mean"].round(4).astype(str) + " +/- " + df_res["precision_se"].round(4).astype(str)
df_res["recall"] = df_res["recall_mean"].round(4).astype(str) + " +/- " + df_res["recall_se"].round(4).astype(str)
df_res["f1"] = df_res["f1_mean"].round(4).astype(str) + " +/- " + df_res["f1_se"].round(4).astype(str)
df_res["time"] = df_res["time_mean"].round(2).astype(str) + " +/- " + df_res["time_se"].round(2).astype(str)

In [8]:
df_res.sort_values(by=["precision_mean", "f1_mean"], ascending=False)

,n_splits,cluster_outliers,precision_mean,precision_se,recall_mean,recall_se,f1_mean,f1_se,time_mean,time_se,splits,precision,recall,f1,time
1,6,all,0.972014,0.000783,0.904537,0.002992,0.937031,0.001426,17.348714,0.098614,None,0.972 +/- 0.0008,0.9045 +/- 0.003,0.937 +/- 0.0014,17.35 +/- 0.1
0,5,all,0.971565,0.000741,0.890830,0.003101,0.929429,0.001938,17.431959,0.071766,None,0.9716 +/- 0.0007,0.8908 +/- 0.0031,0.9294 +/- 0.0019,17.43 +/- 0.07
2,7,all,0.969755,0.000798,0.912548,0.002510,0.940261,0.001222,17.007977,0.045317,None,0.9698 +/- 0.0008,0.9125 +/- 0.0025,0.9403 +/- 0.0012,17.01 +/- 0.05
3,8,all,0.969670,0.000917,0.916313,0.001709,0.942228,0.001025,16.805703,0.043628,None,0.9697 +/- 0.0009,0.9163 +/- 0.0017,0.9422 +/- 0.001,16.81 +/- 0.04
4,9,all,0.969582,0.000643,0.919788,0.001022,0.944024,0.000563,16.895227,0.111103,None,0.9696 +/- 0.0006,0.9198 +/- 0.001,0.944 +/- 0.0006,16.9 +/- 0.11
5,10,all,0.969075,0.000401,0.919402,0.001089,0.943580,0.000498,17.115429,0.043221,None,0.9691 +/- 0.0004,0.9194 +/- 0.0011,0.9436 +/- 0.0005,17.12 +/- 0.04


Observation:

* 1by1 works better with a small splits_percentage
* re-clustering all the outliers significantly increases the precision 
* clustering only the "new" outliers has no noticeable effect on the precision

Conclusion:

The LOW PRECISION is mainly driven by either an identity is not detected thus is
in the outliers (-1) clusters. Or the identity is "detected" but is in a "to be
discarded" clusters, i.e., a large cluster with obvious outliers / multiple
identities. 


* 1by1 is a good solution to add elements to the baseline only if a small number
  of picture is added to the dataset
* clustering only the new outliers is not useful in this case, however we
  believe depeding on the situation it could be useful, e.g., if we add 10+
  pictures of a new identity to the dataset
* This situation may not have been given enough attention in our test bench, and
  this could reflect a real life situation 
* Re-clustering the outliers is necessary if we want to maintain a high precision
  in the baseline

In [9]:
# rename columns cluster_outliers to "OB"
df_res.rename(columns={'cluster_outliers': 'OB', 'time': 'time [s]'}, inplace=True)
print(df_res.sort_values(by=["precision_mean", "f1_mean"], ascending=False).to_latex(index=False, columns=['n_splits', 'OB', 'precision', 'recall', 'f1', 'time [s]']))

\begin{tabular}{rlllll}
\toprule
 n\_splits &  OB &         precision &            recall &                f1 &       time [s] \\
\midrule
        6 & all &  0.972 +/- 0.0008 &  0.9045 +/- 0.003 &  0.937 +/- 0.0014 &  17.35 +/- 0.1 \\
        5 & all & 0.9716 +/- 0.0007 & 0.8908 +/- 0.0031 & 0.9294 +/- 0.0019 & 17.43 +/- 0.07 \\
        7 & all & 0.9698 +/- 0.0008 & 0.9125 +/- 0.0025 & 0.9403 +/- 0.0012 & 17.01 +/- 0.05 \\
        8 & all & 0.9697 +/- 0.0009 & 0.9163 +/- 0.0017 &  0.9422 +/- 0.001 & 16.81 +/- 0.04 \\
        9 & all & 0.9696 +/- 0.0006 &  0.9198 +/- 0.001 &  0.944 +/- 0.0006 &  16.9 +/- 0.11 \\
       10 & all & 0.9691 +/- 0.0004 & 0.9194 +/- 0.0011 & 0.9436 +/- 0.0005 & 17.12 +/- 0.04 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_11071/3363642933.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_res.sort_values(by=["precision_mean", "f1_mean"], ascending=False).to_latex(index=False, columns=['n_splits', 'OB', 'precision', 'recall', 'f1', 'time [s]']))
